In [136]:
import pandas as pd
import yfinance as yf
import ta
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import random
import torch




SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



# Step 1: Download and prepare data
start_date = "2019-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')
df = yf.download("AAPL", start=start_date, end=end_date)

# Flatten column names if they're MultiIndex tuples (e.g., ('Close', 'AAPL'))
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]
df = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
df.dropna(inplace=True)

# Step 2: Add engineered + technical features
try:
    df['Return'] = df['Close'].pct_change()
    df['Candle_Body'] = df['Close'] - df['Open']
    df['Range'] = df['High'] - df['Low']

    df['rsi'] = ta.momentum.RSIIndicator(close=df['Close']).rsi()
    df['macd'] = ta.trend.MACD(close=df['Close']).macd_diff()
    df['ema_10'] = ta.trend.EMAIndicator(close=df['Close'], window=10).ema_indicator()
    df['bb_bbw'] = ta.volatility.BollingerBands(close=df['Close']).bollinger_wband()
    df['adx'] = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close']).adx()
    df['stoch'] = ta.momentum.StochasticOscillator(high=df['High'], low=df['Low'], close=df['Close']).stoch()

    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    df.dropna(inplace=True)
except Exception as e:
    print("Error while adding technical indicators:", e)

print("Final DataFrame shape:", df.shape)

# Step 3: Feature scaling
features = df.columns.tolist()
features.remove('Close')  # Keep 'Close' for labels
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[features])
scaled_close = MinMaxScaler().fit_transform(df[['Close']])
scaled_df = np.concatenate([scaled_close, scaled_features], axis=1)

# Step 4: Create sequences and labels
SEQ_LEN = 30
X, y = [], []

for i in range(SEQ_LEN, len(scaled_df) - 3):
    window = scaled_df[i-SEQ_LEN:i]
    future_price = scaled_df[i+3][0]  # 0 = scaled 'Close'
    current_price = scaled_df[i][0]
    change = future_price - current_price

    threshold = 0.002
    if change > threshold:
        y.append(1)
    elif change < -threshold:
        y.append(0)
    else:
        continue  # Skip small/no change

    X.append(window)

X = np.array(X)
y = np.array(y)

print(f"Final dataset: {X.shape[0]} samples, each of shape {X.shape[1:]}")


/var/folders/l6/kt268nls2sqcx51m4w_z09pc0000gn/T/ipykernel_61694/4293934290.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("AAPL", start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

Final DataFrame shape: (1635, 14)
Final dataset: 1441 samples, each of shape (30, 14)



/var/folders/l6/kt268nls2sqcx51m4w_z09pc0000gn/T/ipykernel_61694/4293934290.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/var/folders/l6/kt268nls2sqcx51m4w_z09pc0000gn/T/ipykernel_61694/4293934290.py:47: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [137]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split

# Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Full dataset
dataset = TensorDataset(X_tensor, y_tensor)

# 80% train, 10% val, 10% test
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

# Loaders
BATCH_SIZE = 64
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

print(f"Train: {len(train_data)} | Val: {len(val_data)} | Test: {len(test_data)}")


Train: 1152 | Val: 144 | Test: 145


In [138]:
import torch.nn as nn
import torch.nn.functional as F

class LSTM_CNN_Attention(nn.Module):
    def __init__(self, input_size, lstm_hidden=64, cnn_out=32, attention_dim=32, num_classes=2):
        super(LSTM_CNN_Attention, self).__init__()
        self.lstm = nn.LSTM(input_size, lstm_hidden, batch_first=True, bidirectional=True)
        
        self.conv1d = nn.Conv1d(in_channels=2 * lstm_hidden, out_channels=cnn_out, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm1d(cnn_out)

        self.attn_fc = nn.Linear(cnn_out, attention_dim)
        self.attn_vector = nn.Linear(attention_dim, 1)

        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(cnn_out, num_classes)

    def forward(self, x):
        # LSTM
        lstm_out, _ = self.lstm(x)  # (B, T, 2*H)

        # CNN
        cnn_input = lstm_out.permute(0, 2, 1)  # (B, 2H, T)
        cnn_out = F.relu(self.bn(self.conv1d(cnn_input)))  # (B, C, T)
        cnn_out = cnn_out.permute(0, 2, 1)  # (B, T, C)

        # Attention
        energy = torch.tanh(self.attn_fc(cnn_out))  # (B, T, A)
        attention_weights = F.softmax(self.attn_vector(energy), dim=1)  # (B, T, 1)
        context = torch.sum(attention_weights * cnn_out, dim=1)  # (B, C)

        # Classification
        out = self.dropout(context)
        out = self.fc(out)  # (B, 2)
        return out


In [139]:
import torch
import torch.nn as nn
import torch.optim as optim

# Instantiate model
model = LSTM_CNN_Attention(input_size=14)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
EPOCHS = 50
best_val_acc = 0
patience = 5
wait = 0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    train_acc = correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            outputs = model(X_val)
            preds = torch.argmax(outputs, dim=1)
            val_correct += (preds == y_val).sum().item()
            val_total += y_val.size(0)

    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {total_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        wait = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered.")
            break


Epoch 1/50 | Train Loss: 12.4605 | Train Acc: 0.5564 | Val Acc: 0.5625
Epoch 2/50 | Train Loss: 12.2515 | Train Acc: 0.5833 | Val Acc: 0.5556
Epoch 3/50 | Train Loss: 12.2778 | Train Acc: 0.5720 | Val Acc: 0.5625
Epoch 4/50 | Train Loss: 12.1429 | Train Acc: 0.5738 | Val Acc: 0.6181
Epoch 5/50 | Train Loss: 12.0462 | Train Acc: 0.5929 | Val Acc: 0.5972
Epoch 6/50 | Train Loss: 11.9956 | Train Acc: 0.6050 | Val Acc: 0.5625
Epoch 7/50 | Train Loss: 12.0721 | Train Acc: 0.5790 | Val Acc: 0.5764
Epoch 8/50 | Train Loss: 11.9015 | Train Acc: 0.6050 | Val Acc: 0.5417
Epoch 9/50 | Train Loss: 12.0407 | Train Acc: 0.5877 | Val Acc: 0.5694
Early stopping triggered.


In [140]:
# Evaluate on test set (no need to load saved model)
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for X_test, y_test in test_loader:
        X_test, y_test = X_test.to(device), y_test.to(device)
        outputs = model(X_test)
        preds = torch.argmax(outputs, dim=1)
        test_correct += (preds == y_test).sum().item()
        test_total += y_test.size(0)

test_acc = test_correct / test_total
print(f"Test Accuracy: {test_acc:.4f}")


Test Accuracy: 0.6276
